In [1]:
# Import needed libraries
import random
from datetime import date, timedelta
import bisect
from typing import List 

# 1. Class Initialization 

In [3]:
class Event:
    def __init__(self, event_id: int, event_name: str, event_date: date, venue: str, total_tickets: int, current_price_per_ticket: float, generated_revenue: float = 0.0):
        """
        Initialize an Event object with details about the event.

        @param event_id: Unique identifier for the event (int)
        @param event_name: Name of the event (str)
        @param event_date: Date of the event (datetime.date)
        @param venue: Location where the event is held (str)
        @param total_tickets: Total number of tickets available for the event (int)
        @param current_price_per_ticket: Current price per ticket for the event (float)
        @param generated_revenue: Total revenue generated from ticket sales (float, default is 0.0)
        """
        self.event_id = event_id
        self.event_name = event_name
        self.event_date = event_date
        self.venue = venue
        self.total_tickets = total_tickets
        self.available_tickets = total_tickets
        self.current_price_per_ticket = current_price_per_ticket
        self.generated_revenue = 0

    
    def check_availability(self):
        """
        Check the number of tickets currently available for the event.

        @return: Number of available tickets (int)
        """
        return self.available_tickets

    def book_tickets(self):
        """
        Book a specified number of tickets for the event.

        @return: True if the booking was successful, False if not enough tickets are available
        """
        if self.available_tickets >= 1:
            self.available_tickets -= 1
            return True
        return False

    def cancel_tickets(self):
        """
        Cancel a specified number of tickets for the event.
        """
        self.available_tickets += 1

    def change_ticket_price(self, new_ticket_price):
        """
        Update the price of tickets for the event.

        @param new_ticket_price: New price per ticket (float)
        """
        self.current_price_per_ticket = new_ticket_price

In [4]:
class User:
    def __init__(self, mit_id: int, name: str, email: str, balance: float = 0.0):
        """
        Initialize a User object to store user details and ticket bookings.

        @param mit_id: Unique identifier for the user (int). Ideally user would need to use Mit_ID for identification
        @param name: Name of the user (str)
        @param email: Email address of the user (str)
        @param balance: Initial account balance for the user (float, default is 0.0)
        """
        self.user_id = mit_id
        self.name = name
        self.email = email
        self.balance = balance
        self.booked_tickets: List[Ticket] = [] # field that will contain all tickets booked from the user by order of date

    def change_email(self, new_email):
        """
        Update the user's email address.

        @param new_email: The new email address to replace the current one (str)
        """
        self.email = new_email

    def balance_pop_up(self, added_amount):
        """
        Add a specified amount to the user's account balance.

        @param added_amount: Amount to add to the current balance (float)
        @return: None, but prints a success message with the updated balance
        """
        if added_amount < 0:
            raise ValueError("Added amount must be greater than zero.")
        self.balance = self.balance + added_amount
        print(f"Balance succesfully popped up by {added_amount}. New balance: {self.balance}") 

    def balance_reduction(self, reduced_amount):
        """
        Reduce the user's account balance by a specified amount.
    
        @param reduced_amount: The amount to be deducted from the user's balance (float)
        @return: None, but prints a success message with the updated balance
        @raises ValueError: If the reduced amount is greater than the available balance
        """
        if reduced_amount > self.balance:
            raise ValueError(f"Insufficient balance. Current balance is {self.balance}, but {reduced_amount} was requested.")
        elif reduced_amount < 0:
            raise ValueError("Reduction amount must be greater than zero.")
        
        self.balance = self.balance - reduced_amount
        print(f"Balance reduced by {reduced_amount}. New balance: {self.balance}") 


In [5]:
class Ticket:
    def __init__(self, ticket_id, user: User, event: Event, booking_date: date, status = "booked"):
        """
        Initialize a new Ticket instance.

        @param ticket_id: Unique identifier for the ticket (int)
        @param user: The User who booked the ticket (User object)
        @param event: The Event for which the ticket is booked (Event object)
        @param booking_date: The date when the ticket was booked (date)
        @param price: The price of the ticket (float)
        @param status: The current status of the ticket (default is "booked")
        """
        self.ticket_id = ticket_id
        self.user = user
        self.event = event
        self.booking_date = booking_date
        self.status = status
        self.price = event.current_price_per_ticket

    def get_ticket_info(self):
        """
        Returns a string with the details of the ticket.

        @return: String representing ticket details
        """
        return f"Ticket {self.ticket_id}: {self.user.name} booked for {self.event.event_name} on {self.event.event_date}"

    def cancel_ticket(self):
        """
        Cancel the ticket and update the event's available tickets.

        @return: None, but updates the ticket status to "canceled" and adjusts event tickets.
        """
        self.status = "canceled"
        self.event.cancel_tickets()  # Cancel 1 ticket for the event

In [6]:
"""
This cell contains two utility functions designed to manage a user's list of tickets in a ticket booking system:

1. `insert_ticket_into_sorted_list`:
   - Purpose: Maintains the sorted order of a user's tickets by `booking_date` when a new ticket is added.
   - Implementation: Utilizes binary search (via `bisect.insort`) to efficiently find the correct position for the new ticket.
   - Use Case: Called whenever a new ticket is booked to ensure that the list of tickets remains sorted in chronological order.
   - Complexity: Efficient for dynamic insertion, with a time complexity of O(n) due to potential shifting of elements.

2. `remove_past_tickets`:
   - Purpose: Removes tickets from a user's list that correspond to past events, leveraging the sorted nature of the list to improve efficiency.
   - Implementation: Iterates through the tickets and retains only those with a `booking_date` on or after the current date.
   - Use Case: Used to clean up a user's ticket list by removing tickets for events that have already occurred.
   - Efficiency: Stops checking as soon as a future ticket is encountered, minimizing unnecessary operations in a sorted list.

Together, these functions ensure that the user's ticket list remains both sorted and up-to-date by automatically handling ticket insertion and cleanup.
"""


def insert_ticket_into_sorted_list(tickets, new_ticket):
    """
    Inserts a ticket into a list of tickets while maintaining the sorted order.

    This function leverages binary search to efficiently find the correct 
    position to insert the new ticket into the list, ensuring that the list 
    remains sorted by the `booking_date` attribute of the tickets.

    @param tickets: A list of Ticket objects, already sorted by booking date (List[Ticket]).
    @param new_ticket: The new Ticket object to be inserted (Ticket).
    @return: None. The input list is updated in place.
    """
    bisect.insort(tickets, new_ticket, key=lambda ticket: ticket.booking_date)

def remove_past_tickets(tickets: List[Ticket]):
    """
    Removes tickets from the list whose booking date is in the past.
    Since the list is sorted, stops as soon as a future ticket is found.

    @param tickets: List of Ticket objects to be checked.
    """
    current_date = date.today()  # Get the current date
    # Use slicing to keep only the tickets that have not passed
    tickets[:] = [ticket for ticket in tickets if ticket.booking_date >= current_date]

# 2. TicketBooking App that cordinates classes 

In [8]:
class TicketBookingApp:
    def __init__(self):
        """
        Initializes a TicketBookingApp instance that manages users and events.
        """
        self.users: Dict[int, User] = {}
        self.events: List[Events] = []
        self.max_user_id = 0
        self.max_event_id = 0
        self.max_ticket_id = 0
        self.current_date = date.today()
    
    def create_user(self, name: str, email: str, balance: float = 0.0):
        """
        Creates a new user and adds them to the system.

        @param name: The name of the user (str)
        @param email: The email address of the user (str)
        @param balance: The initial balance of the user (float, default is 0.0)
        @return: None
        """
        new_user = User(self.max_user_id, name, email, balance)
        self.users[self.max_user_id] = new_user  # Add user with user_id as key
        print(f"User {name} created successfully.")
        self.max_user_id = self.max_user_id + 1
        
    def delete_user(self, user_id):
        """
        Deletes a user by their ID.

        @param user_id: The ID of the user to be deleted (int)
        @return: None
        """
        if user_id in self.users:
            self.users.pop(user_id)
            print(f"User {user_id} removed successfully")
        else:
            print(f"User {user_id} not found!")
    
    def create_event(self,  event_name: str, event_date: date, venue: str, total_tickets: int, current_price_per_ticket: float):
        """
        Creates a new event and adds it to the system.
    
        This function generates a new event object with a unique event ID, provided 
        details (event name, date, venue, ticket count, and ticket price), and adds 
        it to the system's list of events.
    
        @param event_name: The name of the event (str).
        @param event_date: The date of the event (date object).
        @param venue: The location of the event (str).
        @param total_tickets: The total number of tickets available for the event (int).
        @param current_price_per_ticket: The price of one ticket for the event (float).
        @return: None.
        """
        new_event = Event(self.max_event_id, event_name, event_date, venue, total_tickets, current_price_per_ticket)
        self.events.append(new_event)
        print(f"Event {event_name} created successfully.")
        self.max_event_id = self.max_event_id + 1

    def delete_event(self, event_id: int):
        """
        Deletes an event by its unique event ID.
    
        This function searches for an event with the specified event ID in the 
        system's list of events. If found, the event is removed from the list. 
        If the event is not found, a message is displayed, and the function returns 1.
    
        @param event_id: The unique identifier of the event to be deleted (int).
        @return: 0 if the event is successfully deleted, 1 if the event does not exist.
        """
        for event in self.events:
            if event.event_id == event_id: 
                self.events.remove(event)
                return 0
        print(f"Event {event_id} doesn't exist!")
        return 1

    def delete_passed_events(self):
        """
        Deletes events that have passed their scheduled date.
    
        This function iterates through the list of events in the app and removes 
        any event whose event_date is earlier than the provided current_date.
    
        @param current_date: The date to compare events against. Events with 
                             dates before this will be removed (date object).
        @return: None
        """
        for event in self.events:
            if event.event_date < self.current_date: # Check passed events and deletes them from the TicketBookingApp
                self.delete_event(event.event_id)


    def list_events(self):
        """
        Lists all upcoming events and removes past events automatically.
    
        This function first removes all past events (by calling 
        `delete_passed_events`) and then lists the remaining events with their 
        details including event name, date, venue, ticket price, and number of 
        tickets still available.

        @param: None
        @return: None
        """
        self.delete_passed_events()
        if not self.events:
            print("No events available.")
            return
        for event in self.events:
            print(f"{event.event_id}: {event.event_name} on {event.event_date.strftime("%d %B %Y")} at {event.venue}, Price: {event.current_price_per_ticket}, Available Tickets: {event.available_tickets}")

    def book_ticket(self, user_id: int, event_id: int):
        """
        Tries to book a ticket for a user to an event. Checks availability of tickets, 
        user's balance, and ensures the user does not already have a booked ticket 
        for the event.

        @param user_id: The ID of the user booking the ticket (int)
        @param event_id: The ID of the event for which the ticket is being booked (int)
        @return: None
        """
        user = self.users.get(user_id, None)
        event = next((e for e in self.events if e.event_id == event_id), None)
        
        if not user:
            print("User or Event not found.")
            return
        elif not event:
            return

        # Check if the user already has a ticket for the event
        for ticket in user.booked_tickets:
            if ticket.event.event_id == event_id and ticket.status == "booked":
                print(f"Cannot book another ticket. You already have a booked ticket for {event.event_name}.")
                return
        
        # Check if enough tickets are available
        if not event.available_tickets:
            print(f"Currently there are no tickets available.")
            return

        # Check if user has enough balance to book the tickets
        try:
            user.balance_reduction(event.current_price_per_ticket)
        except ValueError as e:
            print(f"Insufficient balance. You need {event.current_price_per_ticket} to book current ticket.")
            return
               
        # Create ticket(s)
        ticket = Ticket(self.max_ticket_id, user = user, event = event, booking_date = date.today())

        # add the new ticket into sorted-by-date list of tickets of the user
        insert_ticket_into_sorted_list(user.booked_tickets, ticket)

        # Increase total revenue of the event
        ticket.event.generated_revenue = ticket.event.generated_revenue + ticket.price
        
        self.max_ticket_id = self.max_ticket_id + 1
        
        # Update event ticket availability and price
        event.available_tickets -= 1
        event.current_price_per_ticket = round(event.current_price_per_ticket + (event.current_price_per_ticket * 0.01), 2)
        
        print(f"Successfully booked a ticket for {event.event_name}.")

    def cancel_ticket(self, user_id: int, ticket_id: int):
        """
        Cancel a user's ticket by ticket_id, update the event's available tickets,
        and increase the user's balance by the ticket price.

        @param user_id: The ID of the user whose ticket is being canceled (int)
        @param ticket_id: The ID of the ticket to be canceled (int)
        @return: None
        """
        # Find the user
        user = self.users.get(user_id, None)
        if not user:
            print("User not found.")
            return
        
        # Find the ticket in the user's booked tickets
        ticket = next((t for t in user.booked_tickets if t.ticket_id == ticket_id), None)
        if not ticket:
            print(f"Ticket {ticket_id} not found for the user.")
            return
        
        # Cancel the ticket
        ticket.cancel_ticket()
        
        # Increase user's balance by the ticket price
        user.balance_pop_up(ticket.price)

        # Decrease total revenue of the event
        ticket.event.generated_revenue = ticket.event.generated_revenue - ticket.price

        # Update event ticket  price
        event.current_price_per_ticket = round(event.current_price_per_ticket - (event.current_price_per_ticket * 0.01), 2)
        
        print(f"Ticket {ticket_id} canceled. User's balance has been updated with the refund of {ticket.price}.")

    def increment_date(self):
        """
        Increments the current date by one day.
        """
        self.current_date += timedelta(days=1)
        print(f"Current date updated to: {self.current_date}")


# 3. Testing functionalities

In [10]:
# Test chunk 1
"""
What It Tests:

Initialization and user creation.
Event creation and past event cleanup.
Successful and failed ticket bookings.
User deletion and verification of remaining users.
Ticket tracking for each user.
Correct event cleanup for past events.
Proper handling of available tickets after bookings.
"""

# Initialize the TicketBookingApp
app = TicketBookingApp()
print("TicketBookingApp initialized.")

# Add users to the app
app.create_user(name="Alice", email="alice@example.com", balance=100.0)
app.create_user(name="Bob", email="bob@example.com", balance=50.0)
app.create_user(name="Charlie", email="charlie@example.com", balance=20.0)

# List the users
print("Users in the system:")
for user_id, user in app.users.items():
    print(f"ID: {user_id}, Name: {user.name}, Email: {user.email}, Balance: {user.balance}")

# Add events to the app
app.create_event(event_name="Concert", event_date=date.today() + timedelta(days=10), venue="Stadium A", total_tickets=1, current_price_per_ticket=30.0)
app.create_event(event_name="Play", event_date=date.today() + timedelta(days=5), venue="Theater B", total_tickets=50, current_price_per_ticket=20.0)
app.create_event(event_name="Past Event", event_date=date.today() - timedelta(days=1), venue="Hall C", total_tickets=30, current_price_per_ticket=15.0)

# List all events (this will also remove past events)
print("Available events:")
app.list_events()

# Book a ticket for Alice to the Concert
app.book_ticket(user_id=0, event_id=0)

# Attempt to book a ticket for Charlie to the Concert
app.book_ticket(user_id=2, event_id=0)

# Delete Bob from the system
app.delete_user(user_id=1)

# Check the remaining users
print("Users after deletion:")
for user_id, user in app.users.items():
    print(f"ID: {user_id}, Name: {user.name}, Email: {user.email}, Balance: {user.balance}")

# Manually list events to see if past events were removed
print("Available events after automatic cleanup:")
app.list_events()

# Book another ticket for Alice to the Play
app.book_ticket(user_id=0, event_id=1)

# Check tickets for Alice
user = app.users[0]  # Retrieve Alice's user object
if user.booked_tickets:  # Check if Alice has any booked tickets
    print(f"Tickets booked by {user.name}:")
    for ticket in user.booked_tickets:
        print(f"Ticket {ticket.ticket_id} for event {ticket.event.event_name} on {ticket.event.event_date}")
else:
    print(f"{user.name} has no booked tickets.")

# Check available tickets for the Play
event = next(e for e in app.events if e.event_id == 1)
print(f"Tickets remaining for the Play: {event.available_tickets}")



TicketBookingApp initialized.
User Alice created successfully.
User Bob created successfully.
User Charlie created successfully.
Users in the system:
ID: 0, Name: Alice, Email: alice@example.com, Balance: 100.0
ID: 1, Name: Bob, Email: bob@example.com, Balance: 50.0
ID: 2, Name: Charlie, Email: charlie@example.com, Balance: 20.0
Event Concert created successfully.
Event Play created successfully.
Event Past Event created successfully.
Available events:
0: Concert on 27 December 2024 at Stadium A, Price: 30.0, Available Tickets: 1
1: Play on 22 December 2024 at Theater B, Price: 20.0, Available Tickets: 50
Balance reduced by 30.0. New balance: 70.0
Successfully booked a ticket for Concert.
Currently there are no tickets available.
User 1 removed successfully
Users after deletion:
ID: 0, Name: Alice, Email: alice@example.com, Balance: 70.0
ID: 2, Name: Charlie, Email: charlie@example.com, Balance: 20.0
Available events after automatic cleanup:
0: Concert on 27 December 2024 at Stadium A,

In [11]:
# Test chunk 2
"""
What the Test Verifies:
Successful Booking: Alice can book the only ticket.
Ticket Availability: Bob is prevented from booking the ticket when it's already sold out.
Ticket Cancellation: Alice can cancel her ticket, which should update her balance and the available tickets for the event.
Rebooking: Bob can successfully book the ticket after Alice cancels her booking, which tests the ability of the system to handle ticket reallocation after cancellations.
Ticket Count Updates: Available tickets should reflect changes after bookings and cancellations.
Revenue Updates: The event’s revenue should be updated accordingly with ticket purchases and cancellations (if you test that part in the actual implementation).
"""
# Initialize the TicketBookingApp
app = TicketBookingApp()
print("TicketBookingApp initialized.")

# Step 1: Create an event with 1 ticket
app.create_event(event_name="Exclusive Concert", event_date=date(2024, 12, 20), 
                 venue="Concert Hall", total_tickets=1, current_price_per_ticket=50.0)

# Step 2: Create two users
app.create_user(name="Alice", email="alice@example.com", balance=100.0)  # Alice can afford a ticket
app.create_user(name="Bob", email="bob@example.com", balance=100.0)  # Bob can also afford a ticket

# Step 3: Alice books the ticket
app.book_ticket(user_id=0, event_id=0)  # Alice books the only available ticket

# Step 4: Bob tries to book the ticket (should fail as it's already booked)
app.book_ticket(user_id=1, event_id=0)  # Bob tries to book the same ticket

# Step 5: Alice cancels the ticket
print("Alice cancels the ticket")
print()
alice = app.users[0]  # Alice's user object
alice.booked_tickets[0].cancel_ticket()  # Cancel Alice's ticket

# Step 6: Bob tries to book the ticket again (should succeed this time)
app.book_ticket(user_id=1, event_id=0)  # Bob tries to book the ticket again

# Check the available tickets for the event
event = next(e for e in app.events if e.event_id == 0)
print(f"Tickets remaining for the Exclusive Concert: {event.available_tickets}")

TicketBookingApp initialized.
Event Exclusive Concert created successfully.
User Alice created successfully.
User Bob created successfully.
Balance reduced by 50.0. New balance: 50.0
Successfully booked a ticket for Exclusive Concert.
Currently there are no tickets available.
Alice cancels the ticket

Balance reduced by 50.5. New balance: 49.5
Successfully booked a ticket for Exclusive Concert.
Tickets remaining for the Exclusive Concert: 0


In [12]:
# Test chunk 3
"""
Key Features Tested:
Ticket booking logic, including availability and user balance checks.
Correct handling of sold-out events.
Balance reduction upon booking and refund upon cancellation.
Rebooking functionality after a ticket is canceled.

This test verifies that the system functions correctly with users booking, canceling, and rebooking 
tickets, with proper balance updates and ticket availability management.
"""

# Initialize the TicketBookingApp
app = TicketBookingApp()

# Create an event with 1 ticket
event_date = date(2024, 12, 15)
app.create_event(event_name="Concert", event_date=event_date, venue="Stadium", total_tickets=1, current_price_per_ticket=100.0)

# Create two users: Alice and Bob
app.create_user(name="Alice", email="alice@example.com", balance=150.0)
app.create_user(name="Bob", email="bob@example.com", balance=50.0)

# Alice books the ticket for the Concert
app.book_ticket(user_id=0, event_id=0)

# Check if Alice's balance has been reduced
alice = app.users[0]
print(f"Alice's balance after booking: {alice.balance}")

# Bob tries to book the ticket for the Concert (should fail as tickets are sold out)
app.book_ticket(user_id=1, event_id=0)

# Now Alice cancels her ticket
app.cancel_ticket(user_id=0, ticket_id=0)

# Check Alice's balance after cancellation
print(f"Alice's balance after canceling: {alice.balance}")

# Now Bob tries to book the ticket again (should succeed after cancellation)
app.book_ticket(user_id=1, event_id=0)

# Check Bob's balance after booking
bob = app.users[1]
print(f"Bob's balance after booking: {bob.balance}")


Event Concert created successfully.
User Alice created successfully.
User Bob created successfully.
Balance reduced by 100.0. New balance: 50.0
Successfully booked a ticket for Concert.
Alice's balance after booking: 50.0
Currently there are no tickets available.
Balance succesfully popped up by 100.0. New balance: 150.0
Ticket 0 canceled. User's balance has been updated with the refund of 100.0.
Alice's balance after canceling: 150.0
Insufficient balance. You need 101.0 to book current ticket.
Bob's balance after booking: 50.0


In [13]:
# Test chunk 4: Sort tickets
"""
Key Features Tested:
Ticket booking logic, including availability checks and user balance validation.
Handling of sold-out events and preventing multiple bookings for the same event when tickets are unavailable.
Balance reduction upon booking a ticket and refund logic upon cancellation.
Rebooking functionality after a ticket is canceled, ensuring that previously booked tickets become available again for other users.

This test verifies that the system correctly manages users booking, canceling, and rebooking tickets, 
while ensuring proper balance updates, ticket availability, and system behavior when events sell out 
or tickets are refunded.
"""

# Hypothetical User and Event (using previously defined classes)
user = User(mit_id=0, name="Alice", email="alice@example.com", balance=100.0)
event = Event(event_id=0, event_name="Concert", event_date=date(2024, 12, 25), venue="Stadium", total_tickets=10, current_price_per_ticket=30.0)

# Create an empty list for tickets
user.booked_tickets = []

# Create shuffled tickets with random booking dates (including past dates)
tickets_to_add = [
    Ticket(ticket_id=0, user=user, event=event, booking_date=date(2024, 12, 15)),
    Ticket(ticket_id=1, user=user, event=event, booking_date=date(2024, 12, 17)),
    Ticket(ticket_id=2, user=user, event=event, booking_date=date(2024, 12, 20)),
    Ticket(ticket_id=3, user=user, event=event, booking_date=date(2024, 12, 18)),
    Ticket(ticket_id=4, user=user, event=event, booking_date=date(2024, 12, 19)),
    Ticket(ticket_id=5, user=user, event=event, booking_date=date(2024, 11, 25)),  # Past date
    Ticket(ticket_id=6, user=user, event=event, booking_date=date(2024, 11, 30)),  # Past date
]

# Add tickets one by one and print after each addition
for ticket in tickets_to_add:
    insert_ticket_into_sorted_list(user.booked_tickets, ticket)
    print(f"Tickets after adding ticket {ticket.ticket_id} (Date: {ticket.booking_date}):")
    for t in user.booked_tickets:
        print(f"Ticket ID: {t.ticket_id}, Booking Date: {t.booking_date}")
    print("-" * 50)

# Now let's remove the past tickets
remove_past_tickets(user.booked_tickets)

# Print the final list of tickets (after removing the past ones)
print("\nTickets after removing past dates:")
for t in user.booked_tickets:
    print(f"Ticket ID: {t.ticket_id}, Booking Date: {t.booking_date}")


Tickets after adding ticket 0 (Date: 2024-12-15):
Ticket ID: 0, Booking Date: 2024-12-15
--------------------------------------------------
Tickets after adding ticket 1 (Date: 2024-12-17):
Ticket ID: 0, Booking Date: 2024-12-15
Ticket ID: 1, Booking Date: 2024-12-17
--------------------------------------------------
Tickets after adding ticket 2 (Date: 2024-12-20):
Ticket ID: 0, Booking Date: 2024-12-15
Ticket ID: 1, Booking Date: 2024-12-17
Ticket ID: 2, Booking Date: 2024-12-20
--------------------------------------------------
Tickets after adding ticket 3 (Date: 2024-12-18):
Ticket ID: 0, Booking Date: 2024-12-15
Ticket ID: 1, Booking Date: 2024-12-17
Ticket ID: 3, Booking Date: 2024-12-18
Ticket ID: 2, Booking Date: 2024-12-20
--------------------------------------------------
Tickets after adding ticket 4 (Date: 2024-12-19):
Ticket ID: 0, Booking Date: 2024-12-15
Ticket ID: 1, Booking Date: 2024-12-17
Ticket ID: 3, Booking Date: 2024-12-18
Ticket ID: 4, Booking Date: 2024-12-19


In [14]:
# Test chunk 5
"""
Key Features Tested:
Proper handling of events with mixed dates (past and future) within the system.
Automatic deletion of past events based on the current date.
Correct listing of events after past events are deleted.

This test verifies that the system accurately identifies and removes past events, 
ensuring that only future events are displayed to the user. It also ensures that the 
delete_passed_events function works as intended by clearing outdated events and updating the 
list of available events accordingly.
"""

# Initialize TicketBookingApp
app = TicketBookingApp()

# Add events with mixed dates (some in the past and some in the future)
app.create_event(event_name="Music Concert", event_date=date(2024, 12, 15), venue="Arena A", total_tickets=100, current_price_per_ticket=50.0)
app.create_event(event_name="Stand-Up Comedy", event_date=date(2024, 12, 10), venue="Club B", total_tickets=50, current_price_per_ticket=30.0)
app.create_event(event_name="Tech Talk", event_date=date(2024, 12, 20), venue="Conference Hall", total_tickets=200, current_price_per_ticket=20.0)
app.create_event(event_name="Art Exhibition", event_date=date(2024, 11, 30), venue="Gallery C", total_tickets=30, current_price_per_ticket=10.0)

# List all events before deleting past events
print("Events before removing past dates:")
app.list_events()

# Remove past events and list remaining events
print("\nRemoving past events...")
app.delete_passed_events()

print("\nEvents after removing past dates:")
app.list_events()


Event Music Concert created successfully.
Event Stand-Up Comedy created successfully.
Event Tech Talk created successfully.
Event Art Exhibition created successfully.
Events before removing past dates:
1: Stand-Up Comedy on 10 December 2024 at Club B, Price: 30.0, Available Tickets: 50
2: Tech Talk on 20 December 2024 at Conference Hall, Price: 20.0, Available Tickets: 200

Removing past events...

Events after removing past dates:
2: Tech Talk on 20 December 2024 at Conference Hall, Price: 20.0, Available Tickets: 200


In [15]:
# Test Case Setup
# Test chunk 6
"""
Key Features Tested:
Booking tickets for events, including successful and unsuccessful attempts based on various conditions (e.g., ticket availability, balance).
Handling of cases where a user tries to book a ticket for an event they already booked.
Verification of behavior when a user has insufficient balance to book a ticket.
Correct failure when attempting to book tickets when no tickets are available for an event.
Restoration of ticket availability and successful booking after updating the available tickets.
Ability for different users to book tickets for the same event.

This test ensures that the system correctly handles multiple booking scenarios, including proper 
validation for ticket availability, user balance, and duplicate bookings. It also tests the 
functionality of booking after restoring ticket availability and the ability of different users 
to successfully book tickets for the same event.
"""

# Create a TicketBookingApp instance
app = TicketBookingApp()

# Create a user
app.create_user(name="Alice", email="alice@example.com", balance=100.0)

# Create an event
app.create_event(event_name="Concert", event_date=date(2024, 12, 25), venue="Stadium", total_tickets=10, current_price_per_ticket=30.0)

# Test 1: User tries to book a ticket for an event (should succeed)
print("\nTest 1: Booking first ticket for Alice")
app.book_ticket(user_id=0, event_id=0)  # Should succeed

# Test 2: User tries to book another ticket for the same event (should fail)
print("\nTest 2: Alice trying to book another ticket for the same event")
app.book_ticket(user_id=0, event_id=0)  # Should fail, already has a booked ticket

# Test 3: User tries to book a ticket with insufficient balance (should fail)
# Change the price for this test case
app.create_event(event_name="Another Concert", event_date=date(2024, 12, 30), venue="Arena", total_tickets=5, current_price_per_ticket=150.0)

print("\nTest 3: Alice trying to book a ticket with insufficient balance")
app.book_ticket(user_id=0, event_id=1)  # Should fail, Alice's balance is 100 and the ticket costs 150

# Test 4: User tries to book a ticket when no tickets are available (should fail)
# Decrease available tickets to 0
event = app.events[0]  # Get the first event (Concert)
event.available_tickets = 0

print("\nTest 4: Alice trying to book a ticket when no tickets are available")
app.book_ticket(user_id=0, event_id=0)  # Should fail, no tickets available

# Test 5: User successfully books a ticket after available tickets are updated
event.available_tickets = 10  # Restore available tickets

print("\nTest 5: Alice successfully booking another ticket")
app.book_ticket(user_id=0, event_id=0)  # Should succeed, enough tickets available

# Test 6: Another user tries to book for the same event (should succeed for a new user)
app.create_user(name="Bob", email="bob@example.com", balance=50.0)
print("\nTest 6: Bob booking a ticket for the concert")
app.book_ticket(user_id=1, event_id=0)  # Should succeed, new user

# Test 7: Alice trying to book again after event is over (should succeed since Alice had no issues before)
print("\nTest 7: Alice trying to book a ticket for another event")
app.book_ticket(user_id=0, event_id=1)  # Should succeed, no issue with the new event


User Alice created successfully.
Event Concert created successfully.

Test 1: Booking first ticket for Alice
Balance reduced by 30.0. New balance: 70.0
Successfully booked a ticket for Concert.

Test 2: Alice trying to book another ticket for the same event
Cannot book another ticket. You already have a booked ticket for Concert.
Event Another Concert created successfully.

Test 3: Alice trying to book a ticket with insufficient balance
Insufficient balance. You need 150.0 to book current ticket.

Test 4: Alice trying to book a ticket when no tickets are available
Cannot book another ticket. You already have a booked ticket for Concert.

Test 5: Alice successfully booking another ticket
Cannot book another ticket. You already have a booked ticket for Concert.
User Bob created successfully.

Test 6: Bob booking a ticket for the concert
Balance reduced by 30.3. New balance: 19.7
Successfully booked a ticket for Concert.

Test 7: Alice trying to book a ticket for another event
Insufficien

In [16]:
# Test chunk 7
"""
Key Features Tested:
Booking tickets: Verifying that users can successfully book tickets for an event and the revenue for the event updates correctly.
Revenue tracking: Ensuring that the event's generated revenue reflects the booking activity.
Ticket cancellation: Testing the cancellation of tickets and ensuring that the revenue decreases appropriately after cancellation.
Refund upon cancellation: Verifying that the user's balance is correctly updated after ticket cancellation, ensuring they are refunded.

This test ensures that the system accurately tracks the revenue generated by ticket sales, 
updates the event’s financials when tickets are booked and canceled, and correctly handles 
refunds to users after cancellations.
"""

app = TicketBookingApp()

# Create some users
app.create_user("Alice", "alice@example.com", 100.0)
app.create_user("Bob", "bob@example.com", 50.0)

# Create an event
event_date = date(2024, 12, 25)
app.create_event("Christmas Concert", event_date, "Stadium", 100, 30.0)

# Print initial state (generated revenue should be 0)
print("Initial Event Revenue:", app.events[0].generated_revenue)

# Book tickets for Alice and Bob
app.book_ticket(0, 0)  # Alice books a ticket for the first event
app.book_ticket(1, 0)  # Bob books a ticket for the same event

# Print updated state (generated revenue should be updated)
print("Updated Event Revenue after bookings:", app.events[0].generated_revenue)

# Cancel a ticket for Alice
ticket_id = app.users[0].booked_tickets[0].ticket_id
app.cancel_ticket(0, ticket_id)  # Alice cancels her ticket

# Print final state (generated revenue should decrease after cancelation)
print("Final Event Revenue after cancellation:", app.events[0].generated_revenue)

# Check Alice's balance after cancelation (should be refunded)
print("Alice's balance after cancellation:", app.users[0].balance)


User Alice created successfully.
User Bob created successfully.
Event Christmas Concert created successfully.
Initial Event Revenue: 0
Balance reduced by 30.0. New balance: 70.0
Successfully booked a ticket for Christmas Concert.
Balance reduced by 30.3. New balance: 19.7
Successfully booked a ticket for Christmas Concert.
Updated Event Revenue after bookings: 60.3
Balance succesfully popped up by 30.0. New balance: 100.0
Ticket 0 canceled. User's balance has been updated with the refund of 30.0.
Final Event Revenue after cancellation: 30.299999999999997
Alice's balance after cancellation: 100.0


In [17]:
# Test chunk 8
"""
Key Features Tested:
Booking tickets with sufficient and insufficient balances: Verifying that users with enough balance (Alice and Bob) can book tickets, while users with insufficient balance (Charlie) cannot.
Revenue tracking: Ensuring the event's generated revenue is updated correctly after successful ticket bookings.
Booking limitations: Confirming that a user (Alice) cannot book multiple tickets for the same event.
Ticket cancellation: Testing the process of ticket cancellation and ensuring the event's generated revenue decreases accordingly, with users being refunded.
Non-existent ticket cancellation: Ensuring that attempting to cancel a non-existent ticket (ticket ID 999) properly fails without affecting the system.
User and event deletion: Verifying that deleting a user (Charlie) and an event (Christmas Concert) updates the system and removes them from the available data.

This test ensures that the system correctly handles ticket bookings, cancellations, and refunds, as 
well as the deletion of users and events, maintaining the integrity of the application.
"""

app = TicketBookingApp()

# Create some users
app.create_user("Alice", "alice@example.com", 100.0)  # Alice has a balance of 100
app.create_user("Bob", "bob@example.com", 50.0)  # Bob has a balance of 50
app.create_user("Charlie", "charlie@example.com", 10.0)  # Charlie has a balance of 10 (insufficient for booking)

# Create an event
event_date = date(2024, 12, 25)
app.create_event("Christmas Concert", event_date, "Stadium", 100, 30.0)  # 100 tickets at 30 each

# Print initial state (generated revenue should be 0)
print("\nInitial Event Revenue:", app.events[0].generated_revenue)

# Book tickets for Alice and Bob (should succeed)
app.book_ticket(0, 0)  # Alice books a ticket for the first event
app.book_ticket(1, 0)  # Bob books a ticket for the same event

# Try to book a ticket for Charlie (should fail due to insufficient balance)
app.book_ticket(2, 0)  # Charlie has only 10.0 in balance, which is insufficient to book a ticket of 30.0

# Print updated state (generated revenue should be updated)
print("\nUpdated Event Revenue after bookings:", app.events[0].generated_revenue)

# Try to book a second ticket for Alice for the same event (should fail since she already has one)
app.book_ticket(0, 0)  # Alice tries to book another ticket for the same event

# Print state after all bookings (to check availability)
app.list_events()  # This should show updated available tickets

# Cancel a ticket for Alice
ticket_id = app.users[0].booked_tickets[0].ticket_id
app.cancel_ticket(0, ticket_id)  # Alice cancels her ticket

# Print final state (generated revenue should decrease after cancellation)
print("\nFinal Event Revenue after cancellation:", app.events[0].generated_revenue)

# Check Alice's balance after cancellation (should be refunded)
print("Alice's balance after cancellation:", app.users[0].balance)

# Try to cancel a ticket for Bob (should succeed)
ticket_id_bob = app.users[1].booked_tickets[0].ticket_id
app.cancel_ticket(1, ticket_id_bob)  # Bob cancels his ticket

# Check Bob's balance after cancellation (should be refunded)
print("Bob's balance after cancellation:", app.users[1].balance)

# Try to cancel a ticket that doesn't exist (should fail)
app.cancel_ticket(0, 999)  # Ticket ID 999 does not exist for Alice

# List events again after cancelations
app.list_events()  # This should show the event with updated available tickets

# Delete a user and check the results
app.delete_user(2)  # Delete Charlie

# Delete an event and check the results
app.delete_event(0)  # Delete the Christmas Concert event

# List events after deletion (should show no events)
app.list_events()


User Alice created successfully.
User Bob created successfully.
User Charlie created successfully.
Event Christmas Concert created successfully.

Initial Event Revenue: 0
Balance reduced by 30.0. New balance: 70.0
Successfully booked a ticket for Christmas Concert.
Balance reduced by 30.3. New balance: 19.7
Successfully booked a ticket for Christmas Concert.
Insufficient balance. You need 30.6 to book current ticket.

Updated Event Revenue after bookings: 60.3
Cannot book another ticket. You already have a booked ticket for Christmas Concert.
0: Christmas Concert on 25 December 2024 at Stadium, Price: 30.6, Available Tickets: 98
Balance succesfully popped up by 30.0. New balance: 100.0
Ticket 0 canceled. User's balance has been updated with the refund of 30.0.

Final Event Revenue after cancellation: 30.299999999999997
Alice's balance after cancellation: 100.0
Balance succesfully popped up by 30.3. New balance: 50.0
Ticket 1 canceled. User's balance has been updated with the refund of 

In [18]:
# Test chunk 9
"""
Key Features Tested:
Handling of past events: Ensuring that events that have already passed are deleted from the system automatically when the current date surpasses the event date.
Booking tickets for future events: Verifying that users can successfully book tickets for future events, while trying to book tickets for past events fails as the event no longer exists.
Revenue tracking for future events: Ensuring that generated revenue is correctly updated after successful bookings for future events.
Ticket cancellation and refund: Testing the cancellation of tickets for future events and ensuring the system correctly updates revenue and user balances after a ticket is canceled.
Deletion of past events and non-existent tickets: Ensuring that the system handles the deletion of past events and prevents users from interacting with deleted events.
User deletion: Verifying that users who haven't booked any tickets (e.g., Charlie) can be deleted without issues.
Event deletion: Ensuring that attempts to delete an already deleted event (such as the past event) do not cause errors.

This test ensures the system correctly manages events based on their date, handling booking, revenue 
updates, ticket cancellations, and deletions of events and users as expected.
"""

# Create a TicketBookingApp instance
app = TicketBookingApp()

# Create some users
app.create_user("Alice", "alice@example.com", 100.0)  # Alice has a balance of 100
app.create_user("Bob", "bob@example.com", 50.0)  # Bob has a balance of 50
app.create_user("Charlie", "charlie@example.com", 10.0)  # Charlie has a balance of 10 (insufficient for booking)

# Create events (one in the past and one in the future)
past_event_date = date(2023, 12, 1)  # A past event (already passed)
future_event_date = date(2024, 12, 25)  # A future event (to be held)

app.create_event("Old Concert", past_event_date, "Old Arena", 50, 30.0)  # Past event with 50 tickets at 30 each
app.create_event("Christmas Concert", future_event_date, "Stadium", 100, 30.0)  # Future event with 100 tickets at 30 each

# Print initial state (generated revenue should be 0 for both)
print("\nInitial Event Revenue for Old Concert:", app.events[0].generated_revenue)
print("Initial Event Revenue for Christmas Concert:", app.events[1].generated_revenue)

# Book tickets for Alice and Bob (should succeed for the future event)
app.book_ticket(0, 1)  # Alice books a ticket for the future event (Christmas Concert)
app.book_ticket(1, 1)  # Bob books a ticket for the future event (Christmas Concert)

# Print updated state (generated revenue should be updated for the future event)
print("\nUpdated Event Revenue for Christmas Concert after bookings:", app.events[1].generated_revenue)

# Now simulate that the current date has moved forward by a day (past event should be deleted)
app.current_date = future_event_date + timedelta(days=1)

# Delete passed events (should delete the past event)
app.delete_passed_events()

# Print event list after passed event deletion
print("\nEvent List after deleting passed events:")
app.list_events()  # Only the future event should be listed, the past event should be removed

# Try to book tickets for Alice and Bob for the past event (should fail)
app.book_ticket(0, 0)  # Alice tries to book a ticket for the past event (should fail as event no longer exists)
app.book_ticket(1, 0)  # Bob tries to book a ticket for the past event (should fail as event no longer exists)

# Cancel a ticket for Alice (should cancel the future event ticket)
ticket_id = app.users[0].booked_tickets[0].ticket_id
app.cancel_ticket(0, ticket_id)  # Alice cancels her ticket for the future event

# Print final state (generated revenue should decrease after cancellation)
# Let's check if the event exists before accessing it:
if len(app.events) > 1:  # Ensure the event list has the expected number of events
    print("\nFinal Event Revenue for Christmas Concert after cancellation:", app.events[1].generated_revenue)
else:
    print("\nError: 'Christmas Concert' event has been deleted unexpectedly!")

# Check Alice's balance after cancellation (should be refunded)
print("Alice's balance after cancellation:", app.users[0].balance)

# Try to cancel a ticket for a past event (should fail because the event no longer exists)
app.cancel_ticket(1, 0)  # Bob tries to cancel a ticket for a past event (should fail)

# List events again after cancellation
app.list_events()  # Should only show the future event, as the past event has been deleted

# Now test deleting users (e.g., delete Charlie, who has not booked any tickets)
app.delete_user(2)  # Charlie has no bookings, so deletion should succeed

# Try to delete the past event (should be already deleted)
app.delete_event(0)  # The "Old Concert" event should already be deleted

# List events again to ensure everything is updated
print("\nEvent List after deleting user and event:")
app.list_events()  # Only the future event should be listed, as the past event was deleted


User Alice created successfully.
User Bob created successfully.
User Charlie created successfully.
Event Old Concert created successfully.
Event Christmas Concert created successfully.

Initial Event Revenue for Old Concert: 0
Initial Event Revenue for Christmas Concert: 0
Balance reduced by 30.0. New balance: 70.0
Successfully booked a ticket for Christmas Concert.
Balance reduced by 30.3. New balance: 19.7
Successfully booked a ticket for Christmas Concert.

Updated Event Revenue for Christmas Concert after bookings: 60.3

Event List after deleting passed events:
No events available.
Balance succesfully popped up by 30.0. New balance: 100.0
Ticket 0 canceled. User's balance has been updated with the refund of 30.0.

Error: 'Christmas Concert' event has been deleted unexpectedly!
Alice's balance after cancellation: 100.0
Ticket 0 not found for the user.
No events available.
User 2 removed successfully
Event 0 doesn't exist!

Event List after deleting user and event:
No events availabl

In [19]:
# Test chunk 10
"""
Key Features Tested:
Verifies that the ticket price increases by 1% when a user successfully books a ticket.
Ensures that the ticket price decreases by 1% when a user cancels a booked ticket.
"""

# Initialize TicketBookingApp
app = TicketBookingApp()

# Create a user with some initial balance
app.create_user("John Doe", "john@example.com", balance=100.0)

# Create an event with an initial ticket price
event_name = "Concert"
event_date = date(2024, 12, 20)
venue = "Madison Square Garden"
total_tickets = 100
initial_price = 50.0
app.create_event(event_name, event_date, venue, total_tickets, initial_price)

# Access the created user and event by their IDs (user_id = 0, event_id = 0)
user_id = 0
event_id = 0

# Print initial ticket price
print(f"Initial ticket price for event '{app.events[event_id].event_name}': {app.events[event_id].current_price_per_ticket}")

# Book a ticket for the user
app.book_ticket(user_id, event_id)

# Print the ticket price after booking (should increase by 1%)
print(f"Ticket price after booking: {app.events[event_id].current_price_per_ticket}")

# Cancel the booked ticket (assuming the ticket_id is 0 for the first booked ticket)
ticket_id = 0  # First ticket booked will have ID 0
app.cancel_ticket(user_id, ticket_id)

# Print the ticket price after cancellation (should decrease by 1%)
print(f"Ticket price after cancellation: {app.events[event_id].current_price_per_ticket}")


User John Doe created successfully.
Event Concert created successfully.
Initial ticket price for event 'Concert': 50.0
Balance reduced by 50.0. New balance: 50.0
Successfully booked a ticket for Concert.
Ticket price after booking: 50.5
Balance succesfully popped up by 50.0. New balance: 100.0
Ticket 0 canceled. User's balance has been updated with the refund of 50.0.
Ticket price after cancellation: 50.5


In [20]:
# Test chunk 10 - Verifying event revenue after bookings and cancellations
"""
Key Features Tested:
- Verifies that the ticket price increases by 1% when a user successfully books a ticket.
- Ensures that the ticket price decreases by 1% when a user cancels a booked ticket.
- Ensures that the revenue of an event is correctly updated after two consecutive bookings by two different users.
"""

# Initialize TicketBookingApp
app = TicketBookingApp()

# Create two users with initial balances
app.create_user("John Doe", "john@example.com", balance=100.0)
app.create_user("Jane Doe", "jane@example.com", balance=100.0)

# Create an event with an initial ticket price
event_name = "Concert"
event_date = date(2024, 12, 20)
venue = "Madison Square Garden"
total_tickets = 100
initial_price = 50.0
app.create_event(event_name, event_date, venue, total_tickets, initial_price)

# Access the created users and event by their IDs (user_id = 0, 1; event_id = 0)
user_id_1 = 0  # John Doe
user_id_2 = 1  # Jane Doe
event_id = 0

# Print initial ticket price and revenue
print(f"Initial ticket price for event '{app.events[event_id].event_name}': {app.events[event_id].current_price_per_ticket}")
print(f"Initial revenue for event '{app.events[event_id].event_name}': {app.events[event_id].generated_revenue}")

# Book the first ticket for the first user (John)
app.book_ticket(user_id_1, event_id)

# Print the ticket price and event revenue after the first booking
print(f"Ticket price after John's booking: {app.events[event_id].current_price_per_ticket}")
print(f"Revenue after John's booking: {app.events[event_id].generated_revenue}")

# Book the second ticket for the second user (Jane)
app.book_ticket(user_id_2, event_id)

# Print the ticket price and event revenue after the second booking
print(f"Ticket price after Jane's booking: {app.events[event_id].current_price_per_ticket}")
print(f"Revenue after Jane's booking: {app.events[event_id].generated_revenue}")


User John Doe created successfully.
User Jane Doe created successfully.
Event Concert created successfully.
Initial ticket price for event 'Concert': 50.0
Initial revenue for event 'Concert': 0
Balance reduced by 50.0. New balance: 50.0
Successfully booked a ticket for Concert.
Ticket price after John's booking: 50.5
Revenue after John's booking: 50.0
Balance reduced by 50.5. New balance: 49.5
Successfully booked a ticket for Concert.
Ticket price after Jane's booking: 51.01
Revenue after Jane's booking: 100.5


In [21]:
# Test chunk 11 - Verifying that date updates correctly
"""
Key Features Tested:
Verifies that the increment_date method correctly updates the current_date by one day for each call.
Ensures that multiple consecutive calls to the increment_date method accurately reflect sequential day increments.
"""

# Initialize TicketBookingApp
app = TicketBookingApp()

# Print the current date initially
print(f"Initial current date: {app.current_date}")

# Call increment_date method to increment the date
app.increment_date()

# Print the updated current date
print(f"Date after 1st increment: {app.current_date}")

# Increment again to verify it works multiple times
app.increment_date()
print(f"Date after 2nd increment: {app.current_date}")


Initial current date: 2024-12-17
Current date updated to: 2024-12-18
Date after 1st increment: 2024-12-18
Current date updated to: 2024-12-19
Date after 2nd increment: 2024-12-19
